In [1]:
import os
if os.getcwd().endswith("experiments"):
    os.chdir("..")

os.getcwd()

'/home/lars/code/python/context-aware-flow-matching'

In [2]:
import torch
from torch.utils.data import random_split
from lightning import seed_everything

import numpy as np
import matplotlib.pyplot as plt

from matplotlib.animation import ArtistAnimation

from pathlib import Path

from tqdm import tqdm, trange

from lightning_trainable.utils import find_checkpoint

import src.utils as U
import src.visualization as viz

from src.datasets import ContextAwareFlowMatchingDataset as CAFMDataset, ModelNet10Dataset
from src.models import Model

In [3]:
torch.autograd.set_grad_enabled(False)

In [4]:
seed_everything(42)

Seed set to 42


42

In [5]:
dataset = ModelNet10Dataset("data/modelnet10")

Found non-empty ModelNet10Dataset in data/modelnet10, skipping download...


Pre-Loading Meshes: 100%|███████████████████████████████████████████████████████████████████| 4899/4899 [00:49<00:00, 98.58it/s]


In [6]:
train_data, val_data, test_data = random_split(dataset, [0.8, 0.1, 0.1])

In [7]:
cp = find_checkpoint()
cp = Path(cp)
cp

PosixPath('lightning_logs/version_10/checkpoints/last.ckpt')

In [8]:
hparams = Model.hparams_type.from_yaml(cp.parent.parent / "hparams.yaml")
model = Model.load_from_checkpoint(cp, hparams=hparams, train_data=train_data, val_data=val_data, test_data=test_data, map_location="cuda:0")
model = model.eval()

In [9]:
# number of shapes to generate an animation for
shapes = 4

# time to transition between distributions
transition_duration = 2

# time to pause at each distribution
pause_duration = 1

# how often to rotate while transitioning
# set to None to disable
# note that with blender-plot 0.0.3, there may be distortions in the resulting animation
rotate_transitions = None

fps = 60
points = 2048
use_blender = True
step_factor = 2

features = 3
steps = fps * transition_duration

In [10]:
samples = torch.stack([model.val_data[i][1] for i in torch.randperm(len(model.val_data))[:shapes]])
samples = samples.to(model.device)
samples = model.reconstruct(samples, integrator="euler", steps=step_factor * steps, progress=True, trajectory=True)[::step_factor]
torch.save(samples, "samples/transition.pt")

1     


In [11]:
samples = torch.load("samples/transition.pt", map_location="cpu")

In [12]:
samples = torch.cat([
    # forward
    samples,
    
    # pause
    U.expand_dim(samples[-1:], 0, fps * pause_duration),
    
    # backward
    samples.flip(0),
    
    # pause
    U.expand_dim(samples[0:1], 0, fps * pause_duration),
])

In [14]:
if rotate_transitions is not None:
    # rotate while transitioning
    samples = U.repeat_dim(samples, 0, rotate_transitions)
    rotations = torch.tensor([0, 0, 360])
    t = torch.linspace(0, 1, samples.shape[0] + 1)[:-1]
    
    rotations = rotations[None, None, None, :] * t[:, None, None, None]
    rotations = rotations.expand_as(samples)
    samples = U.rotate(samples, rotations)

In [15]:
samples = samples.cpu().numpy()

In [16]:
plt.rcParams["animation.ffmpeg_path"] = "/usr/bin/ffmpeg"

for shape in trange(shapes):
    fig = plt.figure(figsize=(4, 4))
    artists = []
    
    for frame in trange(samples.shape[0]):
        if use_blender:
            ax = fig.add_subplot(111)
            artist = viz.scatter_bp(samples[frame, shape], ax=ax)
        else:
            ax = fig.add_subplot(111, projection="3d")
            artist = viz.scatter(samples[frame, shape], ax=ax)

        artists.append([artist])
    
    animation = ArtistAnimation(fig, artists, interval=int(1000 / fps), blit=True)
    print("Saving animation. This may take several minutes...", end=" ")
    animation.save(f"animations/transition_{shape:06d}.mp4", fps=fps, dpi=200)
    print("Done!")
    plt.close()

  0%|                                                                                                   | 0/362 [00:00<?, ?it/s]


  0%|▎                                                                                          | 1/362 [00:03<20:09,  3.35s/it]


  1%|▌                                                                                          | 2/362 [00:06<18:46,  3.13s/it]


  1%|▊                                                                                          | 3/362 [00:09<18:14,  3.05s/it]


  1%|█                                                                                          | 4/362 [00:12<17:58,  3.01s/it]

...


100%|████████████████████████████████████████████████████████████████████████████████████████▊| 361/362 [19:53<00:03,  3.37s/it]


100%|█████████████████████████████████████████████████████████████████████████████████████████| 362/362 [19:56<00:00,  3.31s/it]


Saving animation. This may take several minutes... 

100%|█████████████████████████████████████████████████████████████████████████████████████████| 4/4 [3:06:29<00:00, 2797.32s/it]

Done!
